In [ ]:
import pandas as pd

In [ ]:
auction_user = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_auctions_users.parquet")
auction_results = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_auctions_results.parquet")
user_journey  = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\user_journey_dataset.parquet")
browsing_session = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\browsing_sessions.parquet")
shopping_session = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\shopping_sessions.parquet")
product_catalog = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\processed_sample_catalog.parquet")
impression = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_impressions.parquet")
click = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_clicks.parquet")
purchase = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_purchases.parquet")
auction_user = auction_user.drop_duplicates()

In [82]:
datasets = {
    "auction_user": auction_user,
    "auction_results": auction_results,
    "user_journey": user_journey,
    "browsing_session": browsing_session,
    "shopping_session": shopping_session,
    "product_catalog": product_catalog,
    "impression": impression,
    "click": click,
    "purchase": purchase,
}

for name, df in datasets.items():
    print(f"\n{name} columns:")
    print(df.columns.tolist())



auction_user columns:
['AUCTION_ID', 'OPAQUE_USER_ID', 'CREATED_AT']

auction_results columns:
['AUCTION_ID', 'VENDOR_ID', 'CAMPAIGN_ID', 'PRODUCT_ID', 'RANKING', 'IS_WINNER', 'CREATED_AT']

user_journey columns:
['journey_id', 'auction', 'click', 'impression', 'journey_revenue', 'total_items_purchased', 'did_purchase', 'user_id', 'journey_start_time']

browsing_session columns:
['browsing_session_id', 'user_id', 'shopping_session_id', 'session_start', 'session_end', 'num_events', 'num_auctions', 'num_impressions', 'num_clicks', 'num_purchases', 'session_revenue', 'unique_products', 'unique_auctions', 'duration_minutes', 'session_revenue_usd']

shopping_session columns:
['shopping_session_id', 'user_id', 'shopping_start', 'shopping_end', 'total_events', 'total_auctions', 'total_impressions', 'total_clicks', 'total_purchases', 'total_revenue', 'num_browsing_sessions', 'total_unique_products', 'total_unique_auctions', 'total_browse_minutes', 'shopping_duration_days', 'did_purchase', 'av

In [100]:
print("The auction time is from", auction_results['CREATED_AT'].min(), "to", auction_results['CREATED_AT'].max())
print("The impression time is from ", impression['OCCURRED_AT'].min(), "to", impression['OCCURRED_AT'].max())
print("The purchase time is from ", purchase['PURCHASED_AT'].min(), "to", purchase['PURCHASED_AT'].max())

# find the list of user_id that appears in auctions, note that they may not appear in the impression data
user_ids  = auction_user['OPAQUE_USER_ID'].unique().tolist()

The auction time is from 2025-03-25 00:00:27.680000 to 2025-09-20 23:59:31.906000
The impression time is from  2025-03-25 00:01:05 to 2025-09-20 23:59:56
The purchase time is from  2025-03-25 01:18:17 to 2025-09-20 23:41:05


In [ ]:
# Get unique IDs from both datasets
results_ids = set(auction_results["PRODUCT_ID"].unique())
catalog_ids = set(product_catalog["PRODUCT_ID"].unique())

# Find overlap
overlap_ids = results_ids & catalog_ids

print("Number of unique PRODUCT_IDs in auction_results:", len(results_ids))
print("Number of unique PRODUCT_IDs in product_catalog:", len(catalog_ids))
print("Number of overlapping unique PRODUCT_IDs:", len(overlap_ids))

Number of unique PRODUCT_IDs in auction_results: 3980626
Number of unique PRODUCT_IDs in product_catalog: 3981005
Number of overlapping unique PRODUCT_IDs: 3980421


In [116]:
data = {
    'auction_user': auction_user,
    'auction_results': auction_results,
    'impression': impression,
    'click': click,
    'purchase': purchase
}

df_user = pd.DataFrame(data['auction_user'])
df_results = pd.DataFrame(data['auction_results'])
df_impression = pd.DataFrame(data['impression'])
df_click = pd.DataFrame(data['click'])
df_purchase = pd.DataFrame(data['purchase'])

# --- 2. Data Type Conversion (Time Data) ---
time_cols = ['CREATED_AT']
for df, cols in zip(
    [df_results, df_impression, df_click, df_purchase],
    [time_cols, time_cols, time_cols, ['PURCHASED_AT']]
):
    for col in cols:
        # Check if the column exists and convert
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')


# --- 3. Initial Base Join: auction_results + auction_user ---
# This ensures every auction result is linked to its OPAQUE_USER_ID if available.
df_base = df_results.merge(
    df_user[['AUCTION_ID', 'OPAQUE_USER_ID']],
    on='AUCTION_ID',
    how='left'
)


# --- 4. Funnel Step 1: Base -> Impression (Left Join: Keep all auction results) ---
# Key: AUCTION_ID, PRODUCT_ID
df_funnel_I = df_base.merge(
    df_impression.add_prefix('IMPR_'),
    left_on=['AUCTION_ID', 'PRODUCT_ID'],
    right_on=['IMPR_AUCTION_ID', 'IMPR_PRODUCT_ID'],
    how='left',
    suffixes=('_AUCTION', '_IMPR')
)
# This is the step that introduces the crucial lowercase USER_ID
df_funnel_I.rename(columns={'IMPR_USER_ID': 'USER_ID'}, inplace=True)
df_funnel_I.drop(columns=['IMPR_AUCTION_ID', 'IMPR_PRODUCT_ID'], inplace=True)


# --- 5. Funnel Step 2: Impression -> Click (Left Join) ---
# Key: AUCTION_ID, PRODUCT_ID, USER_ID (The user who saw the impression must be the one who clicked)
df_funnel_IC = df_funnel_I.merge(
    df_click.add_prefix('CLICK_'),
    left_on=['AUCTION_ID', 'PRODUCT_ID', 'USER_ID'],
    right_on=['CLICK_AUCTION_ID', 'CLICK_PRODUCT_ID', 'CLICK_USER_ID'],
    how='left',
    suffixes=('_IMPR', '_CLICK')
)
df_funnel_IC.drop(columns=[
    'CLICK_AUCTION_ID', 'CLICK_PRODUCT_ID', 'CLICK_USER_ID'
], inplace=True)


# --- 6. Funnel Step 3: Click -> Purchase (Left Join with Time Constraint) ---

# Initial left merge on common keys: PRODUCT_ID and USER_ID
df_funnel_ICP = df_funnel_IC.merge(
    df_purchase.add_prefix('PURCH_'),
    left_on=['PRODUCT_ID', 'USER_ID'],
    right_on=['PURCH_PRODUCT_ID', 'PURCH_USER_ID'],
    how='left'
)
df_funnel_ICP.drop(columns=['PURCH_PRODUCT_ID', 'PURCH_USER_ID'], inplace=True)

# Apply the time constraint filter (Purchase must be 0 < time_diff <= 180 minutes after Click)
time_diff = df_funnel_ICP['PURCH_PURCHASED_AT'] - df_funnel_ICP['CLICK_OCCURRED_AT']
time_window = pd.Timedelta(minutes=180)

# Identify rows where a purchase occurred (not NaN) AND the time constraint is NOT met
invalid_purchase_mask = (
    df_funnel_ICP['PURCH_PURCHASED_AT'].notna() &
    (
        # The click time must not be NaN for the comparison
        df_funnel_ICP['CLICK_OCCURRED_AT'].isna() |
        # Purchase must be AFTER the click (time_diff > 0) OR within the window (time_diff <= 180 min)
        (time_diff < pd.Timedelta(seconds=0)) | 
        (time_diff > time_window)
    )
)

# Nullify the purchase-related columns for these invalid rows
purchase_cols = [col for col in df_funnel_ICP.columns if col.startswith('PURCH_')]
df_funnel_ICP.loc[invalid_purchase_mask, purchase_cols] = pd.NA

# Final cleanup and column selection
df_funnel_ICP.rename(
    columns={
        'PURCH_PURCHASED_AT': 'PURCHASE_TIME',
        'PURCH_PURCHASE_ID': 'PURCHASE_ID',
        'PURCH_QUANTITY': 'PURCHASE_QUANTITY',
        'PURCH_UNIT_PRICE': 'PURCHASE_UNIT_PRICE'
    },
    inplace=True
)

# --- 7. Final Funnel Output ---
funnel_cols = [
    'AUCTION_ID', 'PRODUCT_ID', 'VENDOR_ID', 'RANKING', 'IS_WINNER', 'OPAQUE_USER_ID',
    'USER_ID', # The lowercase user ID (will be NaN if no impression occurred)
    'IMPR_INTERACTION_ID', 'IMPR_OCCURRED_AT',
    'CLICK_INTERACTION_ID', 'CLICK_OCCURRED_AT',
    'PURCHASE_ID', 'PURCHASE_TIME', 'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE'
]

df_funnel = df_funnel_ICP[funnel_cols]
df_funnel = df_funnel.drop(columns=["USER_ID"])


print("--- Final Auction Funnel (User-Augmented Base) ---")
print(f"\nTotal Funnel Records: {len(df_funnel)}")
df_funnel.head(10)


--- Final Auction Funnel (User-Augmented Base) ---

Total Funnel Records: 11266574


,AUCTION_ID,PRODUCT_ID,VENDOR_ID,RANKING,IS_WINNER,OPAQUE_USER_ID,IMPR_INTERACTION_ID,IMPR_OCCURRED_AT,CLICK_INTERACTION_ID,CLICK_OCCURRED_AT,PURCHASE_ID,PURCHASE_TIME,PURCHASE_QUANTITY,PURCHASE_UNIT_PRICE
0,068cf3083d327a438a04d657da6b0b99,68b36f76f6035ae220a6c9a0,064b82175e78738fae246401bf837691,43,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
1,068cf3083d327a438a04d657da6b0b99,68464e1521b801733cf51f80,0199482aeacf79a1934852a101ec9cf4,42,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
2,068cf306aed27af4a304a497ed1e4a97,68bc6d1bfe4c68431e101bc6,018f5aaebecb7f38b15a49fb2c2143da,19,True,ext1:c486645f-37aa-4ab1-9493-25cff253a73e,068cf30b-045f-7b00-bf0d-e48aaa317dce,2025-09-20 22:54:31,NaN,NaT,NaN,NaT,NaN,NaN
3,068cf306aed27af4a304a497ed1e4a97,68b39513ff353a5aeb0188b0,0197cb821d2c70519e8c31175ada2394,4,True,ext1:c486645f-37aa-4ab1-9493-25cff253a73e,068cf308-4995-713d-9b0d-d2c4aa317dce,2025-09-20 22:53:40,NaN,NaT,NaN,NaT,NaN,NaN
4,068cf3083d327a438a04d657da6b0b99,68938bad8d99771c81afb287,064befffec19753c9224d9cc25a14230,56,False,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
5,068cf3083d327a438a04d657da6b0b99,68743a849553833d9bbf420a,018fe831606b7c2d8dc927958981f016,40,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
6,068cf3084deb7857b504d2ebaa317dce,68a1d277e48e8638ddbe5c7c,01905bc23cf87fdc91915ea826488366,15,True,ext1:d9e9c8c2-ce45-42a8-96f7-f3a3b0755c07,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
7,068cf3083d327a438a04d657da6b0b99,685af9bf05deb32ffefca8f6,065ceb51222a7fe28f2495ed1739107d,39,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
8,068cf3083d327a438a04d657da6b0b99,68abf64efe4c68018c744a83,0198b6afc6347772b4080dd8ac1daf9a,9,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaN
9,068cf3083d327a438a04d657da6b0b99,68af80b31b40a71c93ac946b,019769a0c9f27a50a41b7c995543f9f9,3,True,ext1:f55df241-e8bf-4497-9b01-b9fb57eb6574,068cf308-bd04-75f6-8a0d-89e37dfaff16,2025-09-20 22:54:02,NaN,NaT,NaN,NaT,NaN,NaN


In [105]:
df_funnel.USER_ID.nunique()

3019

In [118]:
df_funnel['PURCHASE_UNIT_PRICE'].sum()

716700.0

In [108]:
df_funnel.shape

(11266574, 12)